<div style="background-color: #002676; padding: 20px;">
<img src="https://macss.berkeley.edu/wp-content/uploads/2023/09/UCBMaCSS_Logo_2Color_Reverse_TaglineB.png" alt="MaCSS" width="300">
</div>

# **Notebook 4:** Data Analysis

[wdtmacss@berkeley.edu](mailto:wdtmacss@berkeley.edu)\
**Computational Social Science 1A**\
[Human Psychology and Social Technologies](https://classes.berkeley.edu/content/2024-fall-compss-214a-001-lec-001) 
Fall 2024\
UC Berkeley [Masters in Computational Social Science](https://macss.berkeley.edu/about/)

**Week 4:** Performing regressions and DiD analyses in Python.

👩🏾‍🔬🧑‍💻👩🏻‍💻👨🏿‍💻🔬

---

# Table of Contents
1. [Announcements](#announcements)
2. [Summary of Today's Class](#summary-of-todays-class)
3. [Data Simulation](#data-simulation)
    - [Employee Satisfaction Parameters](#employee-satisfaction-parameters)
    - [Time Window Parameters](#time-window-parameters)
    - [Treatment Effect Parameters](#treatment-effect-parameters)
    - [Simulation Loop](#simulation-loop)
4. [Data Analysis](#data-analysis)
    - [Regressions in Python: Some Basics](#regressions-in-python-some-basics)
    - [Outcome Variable: Employee Satisfaction](#outcome-variable-employee-satisfaction)
    - [Categorical Predictor: Company](#categorical-predictor-company)
    - [Continuous Predictor: Time at Company](#continuous-predictor-time-at-company)
    - [Multiple Predictors](#multiple-predictors)
    - [Difference in Differences Analysis](#difference-in-differences-analysis)
        - [Dummy Variables](#dummy-variables)
        - [Interaction Term](#interaction-term)
5. [Data Analysis Challenge 1: Treatment Effect](#data-analysis-challenge-1-treatment-effect)
6. [Data Analysis Challenge 2: Baseline Satisfaction Levels](#data-analysis-challenge-2-baseline-satisfaction-levels)
7. [Assignment Two](#assignment-two)


---

# Announcments
*  **Class structure updates**
    * Only two weeks remaining:
        * previous plan to do a second big dataset seems too rushed now;
        * unfair to request a DiD Twitter analysis assignment this week
    * New schedule structure: focus **entirely on the Twitter dataset**
    * New assignment structure:
        * the twitter analysis is now your **class project**, due at the end of the class
        * we'll have a **scaffolding assignment** before then, on data simulation and analysis of the simulated dataset, designed to really prepare you for DiD analysis before your class project
* **Updated Schedule**:
  * today (data analysis practical session: regression & DiD)
  * next week: scaffolding asignment due (Thursday), class project lab 1 (friday)
  * final week: class project lab 2, plus class discussion of social media perspective papers, farewell cupcackes 🧁🧁
* These updates are captured in the [updated class syllabus](https://bcourses.berkeley.edu/courses/1538139/files/89784577?wrap=1)
*  Questions?

# Summary of Today's Class
The goal today is to build confidence around basic aspects of regression analysis in python, and to develop and understanding of and capacity to implement DiD analysis using regression.  

**Today we will:** 
*  Simulate data using the code we developed last week
*  Perform regression analyses on the simulated data
*  Perform DiD analyses on the simulated data
  

# Data Simulation
Below we will simulate a dataset using the code we developed last week. See `notebook-3-data-simulation.ipynb` for more information on the simulation process and its motivations.

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Employee Satisfaction Parameters

In [7]:
# Mean employee_satisfaction levels for both companies
company_profiles = {
    "Rubicon": {
        "employee_satisfaction": 5.2 # satisfaction on a scale of 1 to 10
    },
    "Giggle": {
        "employee_satisfaction": 6.1 # satisfaction on a scale of 1 to 10
    },
}

# number of  employees we will simulate per company in our simulated dataset
num_employees_per_company = 50

# We will add noise to the employee_satisfaction means when simulating data
# A larger number here means more noise
satisfaction_variance = 1 

## Time window parameters

In [8]:
# Define a data collection time window
data_collection_start_date = pd.to_datetime("2022-01-01")
data_collection_end_date = pd.to_datetime("2023-12-31")

# Create an array of dates on which to simulate collection of satisfaction data
dates = pd.date_range(
    start=data_collection_start_date, 
    end=data_collection_end_date, 
    freq='W' # weekly intervals between start and end
)

## Treatment effect parameters

In [9]:
remote_work_onset_date = pd.to_datetime("2023-01-01")
remote_work_treatment_effect = 2

## Simulation loop

In [10]:
def simulate_data():
    data = []    
    for company, profile in company_profiles.items():
        for date in dates:
            time_at_company = (date - data_collection_start_date).days # this is new
            for i in range(num_employees_per_company):
                satisfaction = np.random.normal(profile["employee_satisfaction"], satisfaction_variance)

                # treatment effect
                if date > remote_work_onset_date:
                    if company == "Giggle":
                        satisfaction += remote_work_treatment_effect 
                
                datapoint = {
                    "date": date, 
                    "company": company,
                    "satisfaction": satisfaction,
                    "employee_id": f"{company[0]}{i}", # create a fake employee id by combining the company name initial letter and the loop index variable
                    "time_at_company": time_at_company
                } 
                data.append(datapoint)
    return pd.DataFrame(data)

In [11]:
# simulate the data
df = simulate_data()

In [12]:
df.sample(n=10)

,date,company,satisfaction,employee_id,time_at_company
4242,2023-08-13,Rubicon,5.807492,R42,589
3052,2023-03-05,Rubicon,3.653969,R2,428
5059,2023-12-10,Rubicon,4.342371,R9,708
5689,2022-02-27,Giggle,5.146953,G39,57
7302,2022-10-16,Giggle,6.215757,G2,288
3086,2023-03-05,Rubicon,6.982606,R36,428
3116,2023-03-12,Rubicon,5.988615,R16,435
4637,2023-10-08,Rubicon,4.467920,R37,645
8998,2023-06-04,Giggle,8.030254,G48,519
8851,2023-05-21,Giggle,7.959903,G1,505


# Data Analysis

## Regressions in Python: some basics
We'll use the [statsmodels](https://www.statsmodels.org/stable/index.html) library to analyze regression models. Statsmodels is nicer than alternatives because it let's you do regressions in python using the same formulae-based approach that is common in R packes such as `lmer`. 

In [13]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

### Outcome variable: employee satisfaction
We're going to try to predict `satisfaction`.

### Categorical predictor: company
First, let's try to predict `satisfaction` from `company`. 

We will express our model using a formula. The structure of the formula is: `dependent variable ~ independent variable`. Here, the tilde sign `~` seperates the outcome (dependent variable, left hand side -- the thing we are trying to predict) from the predictors (independent variable, right hand side). 

In concrete terms, we're trying answer the question: **did employees at the two different companies have different levels of satisfaction?** Since we generated the data ourselves, we know the true answer! 

For this analysis, our formula is:

In [14]:
regression_formula = 'satisfaction ~ company'

We create our model by passing our datset and the formular to statmodels as follows:

In [15]:
model = smf.ols(regression_formula, data=df)

Now that the model is specified, we can *fit* the model to the data as follows:

In [16]:
results = model.fit()

Let's look at the results. I will explain the table in cells below.

In [17]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:           satisfaction   R-squared:                       0.379
Model:                            OLS   Adj. R-squared:                  0.379
Method:                 Least Squares   F-statistic:                     6396.
Date:                Thu, 26 Sep 2024   Prob (F-statistic):               0.00
Time:                        19:38:39   Log-Likelihood:                -16957.
No. Observations:               10500   AIC:                         3.392e+04
Df Residuals:                   10498   BIC:                         3.393e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              7.0933      0

### Interpreting the results table 👀
How does this table help us answer the question? The key section is the middle row, which shows *coefficient estimates* and assosciated statistics. 

#### Coeficient estimates
Here the relevant coeficient estimate is the coeficient for `company`. Recall that the basic purpose of a regression analysis is to estimate the coeficients $w_i$ in an equation of the form:
$$y = x_1 w_1, \ldots, x_n w_n$$
where $x_1,\ldots,x_n$ are predictor variables and $y$ is the outcome variable.  

When we look at the table above, the coefficient estimates are giving us the model's best guess at the $w_i$ terms. So:

*  If the coeficient estaimte for the relevant variable (`company`) is zero, that would tell us that this variable is not predictive of the outcome variable (`satisfaction`), because zero multiplied by anything is equal to zero and therefore **the predictor doesn't influence the outcome** $y$ (here `satisfaction`).
*  If the **coeficient estimate is positive**, that would tell us that the company variable had a positive effect on the outcome variable (i.e. increased `satisfaction`)
    *  **Note**: in the case of binary categorical predictors (`Giggle` vs `Rubicon`), there is generally assumed to be one *reference* class, and the coeficient tells us about the effect of the other class.
    *  Here, the reference class is `Giggle`, and the coeficient tells us about the effect of `Rubicon`. We can see that because the `company` variable in the table states `[T.Rubicon]`
*  If the **coeficient estimate was negative**, we would conclude that the variable had a negative impact relative the the reference class, i.e. working at `Rubicon` is assosciated with a decrease in employee satisfaction relative to `Giggle`.

#### Statistical Significance
You can also look at the colum `P>|t|` to underestand the statistical significance. 

If your alpha value for decidiing statistical statistical significance is `0.05`, you would look at the value in the `P>|t|` column and see if it is **smaller than `0.05`** on the relevant row. If you observe a regression coefficient estimate that is statistically significiantly different than zero, you can conclude that the variable has an effect on `satisfaction` in this dataset. The direction of the effect is given by the sign of the coefficient estimate (`-` or `+`). The magnitude of the effect is a complicated quantity, but is related to the absolute value of the coefficient estimate.

**What does the results table above indicate?**

### Continuous predictor
Let's look at a continuous predictor variable: `time_at_company`. Here, we're actually looking at a **spurious** predicter variable! We know that `time_at_company` is not itself directly causally related to `satisfaction` in a simple way, because we created the data!  

In [18]:
regression_formula = 'satisfaction ~ time_at_company'

In [19]:
model = smf.ols(regression_formula, data=df)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:           satisfaction   R-squared:                       0.072
Model:                            OLS   Adj. R-squared:                  0.072
Method:                 Least Squares   F-statistic:                     816.5
Date:                Thu, 26 Sep 2024   Prob (F-statistic):          5.20e-173
Time:                        19:38:39   Log-Likelihood:                -19061.
No. Observations:               10500   AIC:                         3.813e+04
Df Residuals:                   10498   BIC:                         3.814e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           5.4306      0.029    1

**What do these results indicate?** Does the regression suggest an effect of `time_at_company` on `satisfaction`? If so, why?

### Mulitple predictors
You can include multiple predictors in the model using the formula syntax: `dependent_variable ~ indepdent_variable_1 + independent_variable_2 + ...`.

In [20]:
regression_formula = 'satisfaction ~ company + time_at_company'

In [21]:
model = smf.ols(regression_formula, data=df)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:           satisfaction   R-squared:                       0.451
Model:                            OLS   Adj. R-squared:                  0.451
Method:                 Least Squares   F-statistic:                     4307.
Date:                Thu, 26 Sep 2024   Prob (F-statistic):               0.00
Time:                        19:38:39   Log-Likelihood:                -16309.
No. Observations:               10500   AIC:                         3.262e+04
Df Residuals:                   10497   BIC:                         3.264e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              6.3801      0

**What do these results indicate?** The results of these analyses can be a little unsatisfying or even misleading. This is because the true process that generated the data involves the onset of remote work in our simulation, and this variable has not been included so far. In the next section, we will see how you can isolate the effects of the remote work intervention through the inclusion of an interaction term in the model, implementing a Difference in Differences analysis.

## Difference in Differences Analysis

### Dummy variables

To perform a Difference in Differences analyses that accounts for the onset of a treatment that was applied to one group but not the other, we need to create two binary variables. 

The first binary variable captures **whether the datapoint is before (`0`) or after (`1`) the onset of the treatment**. We can use the `remote_work_onset_date` variable to code this, utilizing the datetime functionality of the `date` column as follows:

In [22]:
df['post_treatment'] = (df['date'] > remote_work_onset_date).astype(int)

The second binary variable captures **whether the employee belongs to the company that offered remote work**. Or in more generic terms, whether the row reflects the group that received the treatment (`1`) or not (`0`). 

In [23]:
# Create the treatment_group variable
df['treatment_group'] = (df['company'] == 'Giggle').astype(int)

In [24]:
df.sample(10)

,date,company,satisfaction,employee_id,time_at_company,post_treatment,treatment_group
7952,2023-01-15,Giggle,7.268299,G2,379,1,1
6848,2022-08-07,Giggle,5.926214,G48,218,0,1
4582,2023-10-01,Rubicon,6.838046,R32,638,1,0
2812,2023-01-29,Rubicon,5.244108,R12,393,1,0
51,2022-01-09,Rubicon,5.750233,R1,8,0,0
9539,2023-08-20,Giggle,8.138399,G39,596,1,1
7510,2022-11-13,Giggle,7.116959,G10,316,0,1
3836,2023-06-18,Rubicon,4.780758,R36,533,1,0
256,2022-02-06,Rubicon,3.868881,R6,36,0,0
7605,2022-11-27,Giggle,6.037988,G5,330,0,1


Together, these two variables can be combined to isolate the effect of the treatment, by incoroporating their *interaction* in the model.

### Interaction Term
Finally, we need to create an interaction term to add to our regression model. An interaction just means that two variables are *multiplied* together. In the case of two binary variables, this means that the resulting product is `1` only if both variables are `1`, and `0` otherwise. 

Here's the regression model we will consider:

*  Dependent variable (outcome): `satisfaction`
*  Independent variable (predictor) 1: `post_treatment`
*  Independent variable (predictor) 2: `treatment_group`
*  Independent variable (predictor) 3: the **interaction** between `post_treatment` and `treatment_group`. The syntax for an interaction term is `post_treatment*treatment_group`.

**Why does the interaction term capture the treatment effect?** Because only the group who received the treatment (remote work) will be coded as `1` under the interaction term.  

So our model formula is:

In [25]:
formula = 'satisfaction ~ post_treatment + treatment_group + post_treatment*treatment_group'

And now we can fit our model:

In [26]:
model = smf.ols(formula, data=df)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:           satisfaction   R-squared:                       0.579
Model:                            OLS   Adj. R-squared:                  0.579
Method:                 Least Squares   F-statistic:                     4819.
Date:                Thu, 26 Sep 2024   Prob (F-statistic):               0.00
Time:                        19:38:40   Log-Likelihood:                -14908.
No. Observations:               10500   AIC:                         2.982e+04
Df Residuals:                   10496   BIC:                         2.985e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept   

**How do you interpret these results?**

## Data Analysis Challenge 1: treatment effect
Regenerate the dataset with **different levels of treatment effect** (the impact of remote work). Change the relevant variable, re-reun the data simulation and analysis, and examine whether the regression results accurately reflect the true effect size. Try making the treatment effect:
*   Stronger
*   Weaker
*   Negative
*   Null (i.e. no effect)

Examine the consequences in the regression results. 

## Data Analysis Challenge 2: baseline satisfaction levels
Edit the **baseline levels of saisfaction** for the two companies. Regenreate the datset a few times with larger or smaller differences in average satisfaction (in the company profiles). Is the regression analysis able to pick up the true effect size of the treatment in a way that is robust to these differences? Can the regression accurately detect the underlying differences in average saistifaction? 

---
# Assignment Two
Assignment two is closely related to the skills and exercises we have covered today in this lab. It is designed to scaffold your understanding of regressions and DiD analyses so that you are prepared to apply these skills in your class project. 

Download and complete the **assignment two notebook** and submit via gradescope **by 10/03/2024 (Thursday next week before midnight Pacific time)**.